In [ ]:
import typing
from automaticTB.examples.structures import get_perovskite_structure
from automaticTB.interaction import InteractionEquation
from automaticTB.printing import print_ao_pairs
from automaticTB.functions import (
    get_namedLCs_from_nncluster, 
    get_free_AOpairs_from_nncluster_and_namedLCs
)

structure = get_perovskite_structure()
print("Solving for the free nearest neighbor interaction in Perovskite")
print("Starting ...")

equation_systems: typing.List[InteractionEquation] = []

for i, nncluster in enumerate(structure.nnclusters):
    print("Cluster centered on " + str(nncluster.crystalsites[nncluster.origin_index]))
    print(nncluster.sitesymmetrygroup.groupname)

    named_lcs = get_namedLCs_from_nncluster(nncluster)
    #for nlc in named_lcs:
    #    print(f"\"{nlc.name}\",")
    #    print(nlc.lc)
    free_pairs = get_free_AOpairs_from_nncluster_and_namedLCs(nncluster, named_lcs)
    print("Solve Interaction ...")
    equation = InteractionEquation.from_nncluster_namedLC(nncluster, named_lcs)

    print_ao_pairs(structure, equation.free_AOpairs)
    equation_systems.append(equation)


In [ ]:
from automaticTB.interaction import CombinedInteractionEquation
from automaticTB.tightbinding import TightBindingModel, gather_InteractionPairs_into_HijRs
from automaticTB.examples.Perovskite.ao_interaction import get_interaction_values_from_list_AOpairs
from automaticTB.printing import print_ao_pairs
import numpy as np

combined_equation = CombinedInteractionEquation(equation_systems)
print_ao_pairs(structure, combined_equation.free_AOpairs)

hvalues = np.array([
    -13.21,    #1 > Free AO interaction: Cl 3s -> Cl 3s @ (  0.00,  0.00,  0.00)
     -9.21,    #2 > Free AO interaction: Pb 6s -> Pb 6s @ (  0.00,  0.00,  0.00)
     -2.16,    #3 > Free AO interaction: Cl 3py -> Cl 3py @ (  0.00,  0.00,  0.00)
      2.39,    #4 > Free AO interaction: Pb 6py -> Pb 6py @ (  0.00,  0.00,  0.00)
     -1.1,     #5 > Free AO interaction: Cl 3s -> Pb 6s @ (  0.00,  0.00,  5.18)
     -0.6,     #6 > Free AO interaction: Cl 3s -> Pb 6pz @ (  0.00,  0.00,  5.18) p0s1
     -1.25,    #7 > Free AO interaction: Cl 3pz -> Pb 6s @ (  0.00,  0.00,  5.18) s0p1
     -3.5,     #8 > Free AO interaction: Cl 3pz -> Pb 6pz @ (  0.00,  0.00,  5.18)
      0.8,     #9 > Free AO interaction: Cl 3px -> Pb 6px @ (  0.00,  0.00,  5.18)
])
hvalues_ref = np.array(
    get_interaction_values_from_list_AOpairs(
        structure.cell, structure.positions, combined_equation.free_AOpairs
    )
)
assert np.allclose(hvalues, hvalues_ref) # ao interaction is correct

solved_interaction = combined_equation.solve_interactions_to_InteractionPairs(hvalues)

hijRs = gather_InteractionPairs_into_HijRs(solved_interaction)
perovskiet_TBmodel = TightBindingModel(
    structure.cell, structure.positions, structure.types, hijRs
)



In [ ]:
from automaticTB.examples.kpaths import cubic_kpath
from automaticTB.properties.bandstructure import BandStructureResult
import os

filename = "HalidePerovskite.pdf"

print("Plot the bandstructure")
band_result = BandStructureResult.from_tightbinding_and_kpath(perovskiet_TBmodel, cubic_kpath)
band_result.plot_data(filename)

os.system(f"rm {filename}") # delete the temporary file